In [24]:
from netCDF4 import Dataset
import numpy as np
from multiprocessing import Process, Manager
import os
from scipy import stats
import cmaps
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cnmaps import draw_maps,get_adm_maps
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import ks_2samp
import scipy.stats as stats
import matplotlib
import random
import time
from tqdm import tqdm
import seaborn as sns
import pandas as pd
from statannotations.Annotator import Annotator
from scipy.stats import genextreme
p_cri=0.01
global_dpi=300

In [25]:
def maskout(data,mask):
    #data is 3d, mask is 2d
    mask_useid=np.where(mask>0,1,np.nan)
    mask_useid=np.expand_dims(mask_useid,axis=0)
    mask_useid=np.repeat(mask_useid,data.shape[0],axis=0)
    data_maskout=data*mask_useid
    data_maskout[:,:,0:70]=np.nan
    return data_maskout

def mask2dout(data,mask):
    #data is 2d, mask is 2d
    mask_useid=np.where(mask>0,1,np.nan)
    data_maskout=data*mask_useid
    data_maskout[:,0:70]=np.nan
    return data_maskout

def cut_edge(data):
    #cut the lateral 7 grids
    #if the data is 3d, then cut the lateral 7 grids
    if len(data.shape)==3:
        data=data[:,7:-7,7:-7]
    #if the data is 2d, then cut the lateral 7 grids
    if len(data.shape)==2:
        data=data[7:-7,7:-7]
    return data

    
def calculate_rmse_pcc(OBS,MOD):
    #OBS and MOD are 2d
    OBS=OBS.flatten()
    MOD=MOD.flatten()
    OBS=OBS[~np.isnan(MOD)]
    MOD=MOD[~np.isnan(MOD)]
    MOD=MOD[~np.isnan(OBS)]
    OBS=OBS[~np.isnan(OBS)]
    rmse=np.sqrt(np.nanmean((OBS-MOD)**2))
    pcc=np.corrcoef(OBS,MOD)[0,1]
    return rmse,pcc

def cleanaligan(OBS,CESM,MPI,CESM_CWRF,MPI_CWRF):
    OBS=np.where(OBS==0,np.nan,OBS)
    CESM=CESM[~np.isnan(OBS)]
    MPI=MPI[~np.isnan(OBS)]
    CESM_CWRF=CESM_CWRF[~np.isnan(OBS)]
    MPI_CWRF=MPI_CWRF[~np.isnan(OBS)]
    OBS=OBS[~np.isnan(OBS)]
    return OBS,CESM,MPI,CESM_CWRF,MPI_CWRF



def ks_boot(arr1,arr2):
    ks_stat,_ = np.round(ks_2samp(arr1,arr2),2)
    return ks_stat

In [26]:
CNregionfil=Dataset("./newmask2.nc")
CNmaskraw=CNregionfil.variables["reg_mask"][:]
CNmaskraw=np.where(CNmaskraw==13,1,CNmaskraw)
CNmaskraw=np.where(CNmaskraw==7,-1,CNmaskraw)
CNmaskraw=np.where(CNmaskraw==8,-1,CNmaskraw)
CNmaskraw=np.where(CNmaskraw==9,-1,CNmaskraw)
CNmaskraw=np.where(CNmaskraw>=7,-1,CNmaskraw)
CNmask=cut_edge(CNmaskraw)

CNregionfil=Dataset("./CN_Subregion_new.nc")
lat2d=np.array(CNregionfil.variables["latitude"][:])
lon2d=np.array(CNregionfil.variables["longitude"][:])
lat2d=cut_edge(lat2d)
lon2d=cut_edge(lon2d)


model=['ERA5','CESM_CWRF','MPI_CWRF','CESM','MPI']
varia=['spi_dry','sti_hot','chd_chd','spei_dry']
types=['freq','inte']
trend_dict={}
infil=Dataset("checkcheck.nc","r")

maskarr=np.array(infil["CESM_ssp585_spei_dry_freq_pvalue"])
maskarr=np.where(~np.isnan(maskarr),1,np.nan)
mask2=np.array(infil["ERA5_hist_chd_chd_freq_trend"])
mask2=np.where(mask2>10e-10,1,np.nan)
maskarr=maskarr*mask2
maskarr=np.where(CNmask>=1,maskarr,np.nan)
xland=np.array(Dataset('wrfinput_d01')['XLAND'][0,:,:])
xland=cut_edge(xland)
maskarr=np.where(xland==1,maskarr,np.nan)


for itype in types:
    for variab in varia:
        for mod in model:
            trend_key=mod+"_hist_"+variab+"_"+itype+"_trend"
            pvalu_key=mod+"_hist_"+variab+"_"+itype+"_pvalue"
            trend_dict[trend_key]=np.array(infil[trend_key][:])*maskarr
            trend_dict[pvalu_key]=np.array(infil[pvalu_key][:])*maskarr
            trend_key=mod+"_hist_"+variab+"_"+itype+"_trend"
            pvalu_key=mod+"_hist_"+variab+"_"+itype+"_pvalue"
            trend_dict[trend_key]=np.array(infil[trend_key][:])*maskarr
            trend_dict[pvalu_key]=np.array(infil[pvalu_key][:])*maskarr

            if not mod == "ERA5":
                for scenario in ['ssp585','ssp245']:
                    trend_key=mod+"_"+scenario+"_"+variab+"_"+itype+"_trend"
                    pvalu_key=mod+"_"+scenario+"_"+variab+"_"+itype+"_pvalue"
                    trend_dict[trend_key]=np.array(infil[trend_key][:])*maskarr
                    trend_dict[pvalu_key]=np.array(infil[pvalu_key][:])*maskarr
                    trend_key=mod+"_"+scenario+"_"+variab+"_"+itype+"_trend"
                    pvalu_key=mod+"_"+scenario+"_"+variab+"_"+itype+"_pvalue"
                    trend_dict[trend_key]=np.array(infil[trend_key][:])*maskarr
                    trend_dict[pvalu_key]=np.array(infil[pvalu_key][:])*maskarr


In [27]:
def draw_mat_hist(freqorinte,scenario,figname,lon,lat,matrix,pvalue,obspvalue,modelname,varname,diff_levs,cm,pcc,rmse,TSval,CNmaskraw,figl,obs_matrix):
    scenario=figname.split("_")[1]
    if scenario=="CWRF":
        scenario=figname.split("_")[2]
    matrix=mask2dout(matrix,CNmask)
    obspvalue=mask2dout(obspvalue,CNmask)
    pvalue=mask2dout(pvalue,CNmask)
    fig=plt.figure(figsize=(5,4))
    proj = ccrs.PlateCarree()  # 创建坐标系
    cwrf_cnproj = ccrs.LambertConformal(central_longitude=110.0, central_latitude=35.17781, false_easting=0.0, false_northing=0.0,  standard_parallels = (30, 60), globe=None, cutoff=-30)
    ax = plt.subplot(1, 1,1, projection=cwrf_cnproj)
    ax.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.3, zorder=99)
    ax.add_feature(cfeat.OCEAN, facecolor="white", zorder=10)
    draw_maps(get_adm_maps(level='国'),color='grey',zorder=99,linewidth=0.8)
    draw_maps(get_adm_maps(level='省'),color='grey',zorder=99,linewidth=0.4)
    ax.set_extent([101,126,16,54])
    # ax.set_extent([100,133,17.2,54])
    cn=ax.pcolormesh(lon,lat,matrix,cmap=cm,vmin=diff_levs[0],vmax=diff_levs[-1],transform = ccrs.PlateCarree())
    # actpvalue=np.where(obspvalue>p_cri,1,pvalue)
    actpvalue=pvalue
    lon_lo=lon[::4,::4]
    lat_lo=lat[::4,::4]
    lon1d=lon_lo.flatten()
    lat1d=lat_lo.flatten()
    actpvalue=actpvalue[::4,::4]
    actpvalue=actpvalue.flatten()
    actlon=lon1d[actpvalue<p_cri]
    actlat=lat1d[actpvalue<p_cri]
    actscatter=ax.scatter(actlon,actlat,marker='.',s=1,color='black',transform = ccrs.PlateCarree())
    if modelname=="ERA5":
        modeldisplay="OBS"
    else:
        modeldisplay=modelname
    ax.text(0.01,0.94,figl+") "+modeldisplay,transform=ax.transAxes,fontsize=10,ha='left',va='bottom',fontweight="bold")

    if scenario=="ssp585":
        matrix_1d=matrix.flatten()
        matrix_1d=matrix_1d[~np.isnan(matrix_1d)]
        pct90=np.percentile(matrix_1d,90)
        print(pct90)
        pct10=np.percentile(matrix_1d,10)
        median=np.median(matrix_1d)

    CNmaskraw_local=cut_edge(CNmaskraw)
    CNmaskraw_local=np.where(CNmaskraw_local==-1,-1,np.nan)
    ax.contourf(lon,lat,CNmaskraw_local,levels=[-1,0],colors='lightgrey',transform = ccrs.PlateCarree(),zorder=100)
    fignamesave = "./final/"+figname+"_"+modelname+"_"+varname+'.png'
    plt.savefig(fignamesave,dpi=global_dpi,bbox_inches='tight')
    os.system("convert -trim "+fignamesave+" "+fignamesave)
    if modelname=="CESM":
        fig=plt.figure(figsize=(1,10))
        cbar_ax = fig.add_axes([0.85, 0.07, 0.1, 0.8])
        cbar=plt.colorbar(cn,cax=cbar_ax,label="",orientation='vertical',shrink=0.02,extend='both')
        cbar.set_ticks(diff_levs[::1])
        cbar.ax.tick_params(labelsize=16)
        if varname=="spei_dry":
            name="SPEI-Dry"
        if varname=="spi_dry":
            name="SPI-Dry"
        if varname=="sti_hot":
            name="STI-Hot"
        if varname=="chd_chd":
            name="CDHE"
        if freqorinte == "freq":
            cbar.set_label("Linear Trend of "+name+" Frequency:[$\mathregular{decade^{-1}}$]",fontsize=16)
        else:
            cbar.set_label("Linear Trend of "+name+" Intensity:[$\mathregular{decade^{-1}}$]",fontsize=16)
        plt.savefig("./final/"+figname+"_"+varname+'diffcolorbar_'+'.png',bbox_inches='tight',dpi=global_dpi)



In [28]:
#draw historical freq trend
seasname=['DJF','MAM','JJA','SON']
figletters=['x']*26
model=['ERA5','CESM_CWRF','MPI_CWRF','CESM','MPI']
figletters=['a','b','c','f','g']

for vid,variab in enumerate(varia):
    for mid,mod in enumerate(model):
        for typeuse in ['freq','inte']:
            for sceid, scenario in enumerate(['hist','ssp585','ssp245']):
                if mod=="ERA5" and scenario!="hist":
                    continue
                trend_key=mod+"_"+scenario+"_"+variab+"_"+typeuse+"_trend"
                pvalu_key=mod+"_"+scenario+"_"+variab+"_"+typeuse+"_pvalue"
                obstr_key="ERA5"+"_hist_"+variab+"_"+typeuse+"_trend"
                obspv_key="ERA5"+"_hist_"+variab+"_"+typeuse+"_pvalue"
                figname=trend_key
                matrix=trend_dict[trend_key]
                pvalue=trend_dict[pvalu_key]
                obs_matrix = trend_dict[obstr_key]
                obspvalue=trend_dict[obspv_key]
                modelname=mod
                varname=variab
                if typeuse == "freq":
                    levels = np.linspace(-0.2,0.2,21)
                else:
                    levels = np.linspace(-0.4,0.4,21)
                cm=cmaps.NCV_blu_red
                pcc,rmse=calculate_rmse_pcc(obs_matrix,matrix)
                TSval=999
                job=Process(target=draw_mat_hist,args=(typeuse,scenario,figname,lon2d,lat2d,matrix,pvalue,obspvalue,modelname,varname,levels,cm,pcc,rmse,TSval,CNmaskraw,figletters[mid],obs_matrix))
                job.start()

job.join()

0.08179622143507004
0.018887895718216894
0.03639798201620579
0.01933845058083534
0.049846287816762924
0.13053263276815413
0.0643921047449112
0.02860067170113325
0.02581350188702345
0.13636364042758942
0.09167764335870743
0.1610671877861023
0.14284145832061768
0.057786154001951216
0.0400746613740921
0.017237592488527298
0.015041721984744072
0.01438296027481556
0.06811911538243294
0.025122442096471787
0.16710583865642548
0.09145805984735489
0.025472111999988556
0.131242236495018
0.06794624626636506
0.03876723125576973
0.09716732800006866
0.1445631429553032
0.12257463186979294
0.0555555559694767
0.04190821237862108
0.03314808718860149


In [29]:
def moca(GCM,sizeuse):
    GCM=GCM[~np.isnan(GCM)]
    n_iterations = 10000
    GCM_means = np.zeros(n_iterations)
    for i in range(n_iterations):
        GCM_sample = np.random.choice(GCM, sizeuse, replace=True)
        GCM_means[i] = np.mean(GCM_sample)
    GCM_ci = np.percentile(GCM_means, [0.5, 99.5])
    GCM_mean = np.mean(GCM_means)
    return GCM_mean, GCM_ci[0], GCM_ci[1]

def kde_mont_draw_hist_original(figname,scenario,varname,OBS,CESM,MPI,CESM_CWRF,MPI_CWRF):
    OBS,CESM,MPI,CESM_CWRF,MPI_CWRF=cleanaligan(OBS,CESM,MPI,CESM_CWRF,MPI_CWRF)
    OBS=OBS.flatten()
    CESM=CESM.flatten()
    MPI=MPI.flatten()
    CESM_CWRF=CESM_CWRF.flatten()
    MPI_CWRF=MPI_CWRF.flatten()
    fig, axs = plt.subplots(2, 1, gridspec_kw={'height_ratios': [2, 1]}, sharex=True,figsize=(6.35,9.29))
    ax=axs[0]
    if scenario=="hist":
        CESM_ks_stat_mean = ks_boot(OBS, CESM)
        CESM_CWRF_ks_stat_mean = ks_boot(OBS, CESM_CWRF)
        MPI_ks_stat_mean = ks_boot(OBS, MPI)
        MPI_CWRF_ks_stat_mean = ks_boot(OBS, MPI_CWRF)
        ERA5kde=sns.kdeplot(ax=ax,x=OBS,fill=True,color='black',lw=2.4,ls='-',label='OBS')
        CESM_kde=sns.kdeplot(ax=ax,x=CESM,fill=False,color='pink',lw=2.4,ls='-',label='CESM(KS:'+str(CESM_ks_stat_mean)+')')
        CESM_CWRF_kde=sns.kdeplot(ax=ax,x=CESM_CWRF,fill=False,color='red',lw=3,ls='--',label='CESM_CWRF(KS:'+str(CESM_CWRF_ks_stat_mean)+')')
        MPI_kde=sns.kdeplot(ax=ax,x=MPI,fill=False,color='lightblue',lw=2.4,ls='-',label='MPI(KS:'+str(MPI_ks_stat_mean)+')')
        MPI_CWRF_kde=sns.kdeplot(ax=ax,x=MPI_CWRF,fill=False,color='blue',lw=3,ls='--',label='MPI_CWRF(KS:'+str(MPI_CWRF_ks_stat_mean)+')')
    else:
        CESM_kde=sns.kdeplot(ax=ax,x=CESM,fill=False,color='pink',lw=2.4,ls='-',label='CESM')
        CESM_CWRF_kde=sns.kdeplot(ax=ax,x=CESM_CWRF,fill=False,color='red',lw=3,ls='--',label='CESM_CWRF')
        MPI_kde=sns.kdeplot(ax=ax,x=MPI,fill=False,color='lightblue',lw=2.4,ls='-',label='MPI')
        MPI_CWRF_kde=sns.kdeplot(ax=ax,x=MPI_CWRF,fill=False,color='blue',lw=3,ls='--',label='MPI_CWRF')
    ax.legend(loc=[0.01,0.62],fontsize=13,frameon=False)
    ax.text(0.02,0.97,"d)",transform=ax.transAxes,fontsize=20,ha='left',va='top',fontweight="bold",zorder=100)
    ax.axvline(0,zorder=1, color='black', linewidth=1,ls='--')
    if varname=="spei_dry":
        name="SPEI-Dry"
    if varname=="spi_dry":
        name="SPI-Dry"
    if varname=="sti_hot":
        name="STI-Hot"
    if varname=="chd_chd":
        name="CDHE"
    ax.set_ylabel("Density",fontsize=13)
    ax_box=axs[1]
    sizeuse=100
    OBS_mean,OBS_low,OBS_high = moca(OBS,sizeuse)
    CESM_mean,CESM_low,CESM_high = moca(CESM,sizeuse)
    MPI_mean,MPI_low,MPI_high = moca(MPI,sizeuse)
    CESM_CWRF_mean,CESM_CWRF_low,CESM_CWRF_high = moca(CESM_CWRF,sizeuse)
    MPI_CWRF_mean,MPI_CWRF_low,MPI_CWRF_high = moca(MPI_CWRF,sizeuse)
    colorpick=['black','pink','red','lightblue','blue']
    labelpick=['OBS','CESM','CESM_CWRF','MPI','MPI_CWRF']
    for mid, label in enumerate(labelpick):
        ypos=0-mid
        colorlocal=colorpick[mid] 
        C,L,H = eval(label+"_mean"),eval(label+"_low"),eval(label+"_high")     
        ax_box.plot([L,L],[ypos-0.5,ypos+0.5], color=colorlocal,zorder=10, linewidth=1.5)
        ax_box.plot([H,H],[ypos-0.5,ypos+0.5], color=colorlocal,zorder=10, linewidth=1.5)
        ax_box.plot([C], [ypos], color=colorlocal,zorder=10, marker='o', markersize=4.5)
    ax_box.set_ylim(-5,1)
    ax_box.set_yticks([0,-1,-2,-3,-4])
    ax_box.set_yticklabels(['OBS','CESM','CESM_CWRF','MPI','MPI_CWRF'],fontsize=14)
    xticks=ax_box.get_xticks()
    if scenario=="hist":
        if figname=="freq":
            if varname=="spei_dry":
                ax_box.set_xlim(-0.12,0.11)
            if varname=="sti_hot":
                ax_box.set_xlim(-0.08,0.16)
            if varname=="chd_chd":
                ax_box.set_xlim(-0.07,0.134)

    ax_box.text(0.02,0.97,"e)",transform=ax_box.transAxes,fontsize=20,ha='left',va='top',fontweight="bold",zorder=100)

    ax_box.axvline(0,zorder=1, color='black', linewidth=1,ls='--')
    if figname=="freq":
        ax_box.set_xlabel("Linear Trend of "+name+" Frequency:[$\mathregular{decade^{-1}}$]",fontsize=13)
    else:
        ax_box.set_xlabel("Linear Trend of "+name+" Intensity:[$\mathregular{decade^{-1}}$]",fontsize=13)
    ax.set_ylabel("Kernel Density Estimation",fontsize=15)
    plt.subplots_adjust(hspace=0.05)
    plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)
    plt.savefig('./final/Original_KDE'+figname+"_"+scenario+"_"+varname+'.png',dpi=global_dpi,bbox_inches='tight')
    os.system("convert -trim ./final/Original_KDE"+figname+"_"+scenario+"_"+varname+'.png ./final/Original_KDE'+figname+"_"+scenario+"_"+varname+'.png')


for vid,variab in enumerate(varia):
    for typeuse in ['freq','inte']:
        for sceid, scenario in enumerate(['hist']):
            OBS=trend_dict["ERA5_hist_"+variab+"_"+typeuse+"_trend"]
            CESM=trend_dict["CESM_"+scenario+"_"+variab+"_"+typeuse+"_trend"]
            MPI=trend_dict["MPI_"+scenario+"_"+variab+"_"+typeuse+"_trend"]
            CESM_CWRF=trend_dict["CESM_CWRF_"+scenario+"_"+variab+"_"+typeuse+"_trend"]
            MPI_CWRF=trend_dict["MPI_CWRF_"+scenario+"_"+variab+"_"+typeuse+"_trend"]
            job=Process(target=kde_mont_draw_hist_original,args=(typeuse,scenario,variab,OBS,CESM,MPI,CESM_CWRF,MPI_CWRF))
            job.start()
job.join()

In [69]:
def motencarlo_bias(OBS,CESM,CESM_CWRF,MPI,MPI_CWRF):
    CESM_bias = np.ndarray([10000])
    CESM_CWRF_bias = np.ndarray([10000])
    MPI_bias = np.ndarray([10000])
    MPI_CWRF_bias = np.ndarray([10000])

    for i in range(10000):
        indices = random.choices(range(len(OBS)), k=100)
        OBS_sample = OBS[indices]
        CESM_sample = CESM[indices]
        MPI_sample = MPI[indices]
        CESM_CWRF_sample = CESM_CWRF[indices]
        MPI_CWRF_sample = MPI_CWRF[indices]
        CESM_bias[i] = np.nanmean(CESM_sample - OBS_sample)
        CESM_CWRF_bias[i] = np.nanmean(CESM_CWRF_sample - OBS_sample)
        MPI_bias[i] = np.nanmean(MPI_sample - OBS_sample)
        MPI_CWRF_bias[i] = np.nanmean(MPI_CWRF_sample - OBS_sample)
    return CESM_bias, CESM_CWRF_bias, MPI_bias, MPI_CWRF_bias

def motencarlo_rmse(OBS,CESM,CESM_CWRF,MPI,MPI_CWRF):
    CESM_rmse = np.ndarray([10000])
    CESM_CWRF_rmse = np.ndarray([10000])
    MPI_rmse = np.ndarray([10000])
    MPI_CWRF_rmse = np.ndarray([10000])
    for i in range(10000):
        indices = random.choices(range(len(OBS)), k=100)
        OBS_sample = OBS[indices]
        CESM_sample = CESM[indices]
        MPI_sample = MPI[indices]
        CESM_CWRF_sample = CESM_CWRF[indices]
        MPI_CWRF_sample = MPI_CWRF[indices]
        CESM_rmse[i] = np.sqrt(np.nanmean((CESM_sample - OBS_sample) ** 2))
        CESM_CWRF_rmse[i] = np.sqrt(np.nanmean((CESM_CWRF_sample - OBS_sample) ** 2))
        MPI_rmse[i] = np.sqrt(np.nanmean((MPI_sample - OBS_sample) ** 2))
        MPI_CWRF_rmse[i] = np.sqrt(np.nanmean((MPI_CWRF_sample - OBS_sample) ** 2))
    return CESM_rmse, CESM_CWRF_rmse, MPI_rmse, MPI_CWRF_rmse

def get_mean_99CI(data):
    data = data[~np.isnan(data)]
    mean = np.mean(data)
    low = np.percentile(data, 0.5)
    high = np.percentile(data, 99.5)
    return mean, low, high

# bootstrapping_new(variab, samples, size,
def bootstrap_RMSE_BIASES(variab, samples, size,
                  OBS_matrix,
                  CESM_matrix, MPI_matrix,
                  CESM_CWRF_matrix, MPI_CWRF_matrix):

    # cesm_cwrf_pointwise_bias = cesm_cwrf_matrix - obs_matrix
    # mpi_cwrf_pointwise_bias = mpi_cwrf_matrix - obs_matrix
    # cesm_pointwise_bias = cesm_matrix - obs_matrix
    # mpi_pointwise_bias = mpi_matrix - obs_matrix
    # sizeuse=100
    # CESM_mean,CESM_low,CESM_high = moca(cesm_pointwise_bias,sizeuse)
    # CESM_CWRF_mean,CESM_CWRF_low,CESM_CWRF_high = moca(cesm_cwrf_pointwise_bias,sizeuse)
    # MPI_mean,MPI_low,MPI_high = moca(mpi_pointwise_bias,sizeuse)
    # MPI_CWRF_mean,MPI_CWRF_low,MPI_CWRF_high = moca(mpi_cwrf_pointwise_bias,sizeuse)


    CESM_bias, CESM_CWRF_bias, MPI_bias, MPI_CWRF_bias = motencarlo_bias(obs_matrix,cesm_matrix,cesm_cwrf_matrix,mpi_matrix,mpi_cwrf_matrix)
    CESM_mean, CESM_low, CESM_high = get_mean_99CI(CESM_bias)
    CESM_CWRF_mean, CESM_CWRF_low, CESM_CWRF_high = get_mean_99CI(CESM_CWRF_bias)
    MPI_mean, MPI_low, MPI_high = get_mean_99CI(MPI_bias)
    MPI_CWRF_mean, MPI_CWRF_low, MPI_CWRF_high = get_mean_99CI(MPI_CWRF_bias)

    fig = plt.figure(figsize=(8,3))
    ax_box = plt.subplot(1, 1, 1)
    colorpick=['pink','red','lightblue','blue']
    labelpick=['CESM','CESM_CWRF','MPI','MPI_CWRF']
    for mid, label in enumerate(labelpick):
        ypos=0-mid
        colorlocal=colorpick[mid] 
        C,L,H = eval(label+"_mean"),eval(label+"_low"),eval(label+"_high")     
        ax_box.plot([L,L],[ypos-0.5,ypos+0.5], color=colorlocal,zorder=10, linewidth=1.5)
        ax_box.plot([H,H],[ypos-0.5,ypos+0.5], color=colorlocal,zorder=10, linewidth=1.5)
        ax_box.plot([C], [ypos], color=colorlocal,zorder=10, marker='o', markersize=4.5)
    ax_box.set_ylim(-4,1)
    ax_box.set_yticks([0,-1,-2,-3])
    ax_box.set_yticklabels(['CESM','CESM_CWRF','MPI','MPI_CWRF'],fontsize=14)
    ax_box.axvline(0,zorder=1, color='black', linewidth=2,ls='--')
    xticks=ax_box.get_xticks()
    ax_box.set_xlim(-0.06,0.03)
    ax_box.set_xlabel("Bias [$\mathregular{decade^{-1}}$]",fontsize=14)
    if variab == "sti_hotfreq":
        figl = "a"
    if variab == "spei_dryfreq":
        figl = "c"
    if variab == "chd_chdfreq":
        figl = "e"
    ax_box.text(-0.18,0.97,figl+")",transform=ax_box.transAxes,fontsize=20,ha='left',va='top',fontweight="bold")
    # plt.savefig('./test.png',bbox_inches='tight',dpi=global_dpi)
    plt.savefig("final/Down_box_" + variab + "_Bias" + ".png", dpi=300, bbox_inches='tight')



    CESM_rmse, CESM_CWRF_rmse, MPI_rmse, MPI_CWRF_rmse = motencarlo_rmse(obs_matrix,cesm_matrix,cesm_cwrf_matrix,mpi_matrix,mpi_cwrf_matrix)
    CESM_mean, CESM_low, CESM_high = get_mean_99CI(CESM_rmse)
    CESM_CWRF_mean, CESM_CWRF_low, CESM_CWRF_high = get_mean_99CI(CESM_CWRF_rmse)
    MPI_mean, MPI_low, MPI_high = get_mean_99CI(MPI_rmse)
    MPI_CWRF_mean, MPI_CWRF_low, MPI_CWRF_high = get_mean_99CI(MPI_CWRF_rmse)

    fig = plt.figure(figsize=(8,3))
    ax_box = plt.subplot(1, 1, 1)
    colorpick=['pink','red','lightblue','blue']
    labelpick=['CESM','CESM_CWRF','MPI','MPI_CWRF']
    for mid, label in enumerate(labelpick):
        ypos=0-mid
        colorlocal=colorpick[mid] 
        C,L,H = eval(label+"_mean"),eval(label+"_low"),eval(label+"_high")     
        ax_box.plot([L,L],[ypos-0.5,ypos+0.5], color=colorlocal,zorder=10, linewidth=1.5)
        ax_box.plot([H,H],[ypos-0.5,ypos+0.5], color=colorlocal,zorder=10, linewidth=1.5)
        ax_box.plot([C], [ypos], color=colorlocal,zorder=10, marker='o', markersize=4.5)
    ax_box.set_ylim(-4,1)
    ax_box.set_yticks([0,-1,-2,-3])
    ax_box.set_yticklabels(['CESM','CESM_CWRF','MPI','MPI_CWRF'],fontsize=14)
    ax_box.axvline(0,zorder=1, color='black', linewidth=2,ls='--')
    xticks=ax_box.get_xticks()
    ax_box.set_xlim(0,0.046)
    ax_box.set_xlabel("RMSE [$\mathregular{decade^{-1}}$]",fontsize=14)
    if variab == "sti_hotfreq":
        figl = "b"
    if variab == "spei_dryfreq":
        figl = "d"
    if variab == "chd_chdfreq":
        figl = "f"
    ax_box.text(-0.18,0.97,figl+")",transform=ax_box.transAxes,fontsize=20,ha='left',va='top',fontweight="bold")
    plt.savefig("final/Down_box_" + variab + "_RMSE" + ".png", dpi=300, bbox_inches='tight')




for vid,variab in enumerate(['sti_hot','spei_dry','chd_chd']):
    for typeuse in ['freq']:
        sernario='hist'
        obs_matrix = trend_dict["ERA5_hist_"+variab+"_"+typeuse+"_trend"]
        cesm_matrix = trend_dict["CESM_hist_"+variab+"_"+typeuse+"_trend"]
        mpi_matrix = trend_dict["MPI_hist_"+variab+"_"+typeuse+"_trend"]
        cesm_cwrf_matrix = trend_dict["CESM_CWRF_hist_"+variab+"_"+typeuse+"_trend"]
        mpi_cwrf_matrix = trend_dict["MPI_CWRF_hist_"+variab+"_"+typeuse+"_trend"]
        job=Process(target=bootstrap_RMSE_BIASES,args=(variab+typeuse,10000,100,obs_matrix,cesm_matrix,mpi_matrix,cesm_cwrf_matrix,mpi_cwrf_matrix))
        job.start()
    
job.join()

!convert +append ./final/Down_box_sti_hotfreq_Bias.png ./final/Down_box_sti_hotfreq_RMSE.png ./final/Down_box_sti_hot.png
!convert +append ./final/Down_box_spei_dryfreq_Bias.png ./final/Down_box_spei_dryfreq_RMSE.png ./final/Down_box_spei_dry.png
!convert +append ./final/Down_box_chd_chdfreq_Bias.png ./final/Down_box_chd_chdfreq_RMSE.png ./final/Down_box_chd_chd.png
!convert -append ./final/Down_box_sti_hot.png ./final/Down_box_spei_dry.png ./final/Down_box_chd_chd.png ./final/Down_box.png


In [61]:
!convert +append ./final/Down_box_sti_hotfreq_Bias.png ./final/Down_box_sti_hotfreq_RMSE.png ./final/Down_box_sti_hot.png
!convert +append ./final/Down_box_spei_dryfreq_Bias.png ./final/Down_box_spei_dryfreq_RMSE.png ./final/Down_box_spei_dry.png
!convert +append ./final/Down_box_chd_chdfreq_Bias.png ./final/Down_box_chd_chdfreq_RMSE.png ./final/Down_box_chd_chd.png
!convert -append ./final/Down_box_sti_hot.png ./final/Down_box_spei_dry.png ./final/Down_box_chd_chd.png ./final/Down_box.png


In [35]:
def mergepic(varname,mode):
    os.system("convert +append ./final/ERA5_hist_"+varname+"_"+mode+"_trend_ERA5_"+varname+".png ./final/CESM_CWRF_hist_"+varname+"_"+mode+"_trend_CESM_CWRF_"+varname+".png ./final/MPI_CWRF_hist_"+varname+"_"+mode+"_trend_MPI_CWRF_"+varname+".png ./final/"+mode+"_"+varname+"_lineone.png")
    os.system("convert -resize 796x987! ./final/Original_KDE"+mode+"_hist_"+varname+".png ./final/"+mode+"_"+varname+"_leftdown.png")
    os.system("convert +append ./final/"+mode+"_"+varname+"_leftdown.png ./final/CESM_hist_"+varname+"_"+mode+"_trend_CESM_"+varname+".png ./final/MPI_hist_"+varname+"_"+mode+"_trend_MPI_"+varname+".png ./final/"+mode+"_"+varname+"_linetwo.png")
    os.system("convert -gravity southeast -append ./final/"+mode+"_"+varname+"_lineone.png ./final/"+mode+"_"+varname+"_linetwo.png ./final/"+mode+"_"+varname+"left.png")
    os.system("convert -resize x1858 ./final/CESM_hist_"+varname+"_"+mode+"_trend_"+varname+"diffcolorbar_.png ./final/"+mode+"_"+varname+"right.png")
    os.system("convert -gravity northeast +append ./final/"+mode+"_"+varname+"left.png ./final/"+mode+"_"+varname+"right.png ./"+mode+"_"+varname+"_final.png")

for varname in varia:
    for mode in ['freq','inte']:
        job = Process(target=mergepic,args=(varname,mode))
        job.start()
job.join()